# Face Mask Detection (CNN, ResNet50)

Bulding CNN and ResNet50 for Face Mask Detection

Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import os 
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from keras.applications import ResNet50
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from keras.regularizers import l2
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ReduceLROnPlateau
import cv2
import pathlib
import random

In [ ]:
dataset_path = pathlib.Path('../input/face-mask-12k-images-dataset/Face Mask Dataset')
Test_path = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Test'
Train_path = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Train'
Val_path = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation'

images=os.listdir(Train_path)
images

Showing image with mask

In [ ]:
Mask_img =  mpimg.imread('../input/face-mask-12k-images-dataset/Face Mask Dataset/Train/WithMask/272.png')
plt.imshow(Mask_img)
plt.title("person with mask")

Showing image without mask

In [ ]:
Mask_img =  mpimg.imread('../input/face-mask-12k-images-dataset/Face Mask Dataset/Train/WithoutMask/14.png')
plt.imshow(Mask_img)
plt.title("person without mask")

Preparing image data, splitting into train, cross validation, test sets 

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range=0.2)

val_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2)

test_datagen = ImageDataGenerator(rescale = 1./255)  #Image normalization.

training_set = train_datagen.flow_from_directory(Train_path,
                                                 target_size = (128, 128),
                                                 interpolation="nearest",
                                                 class_mode='binary',
                                                 classes=["WithoutMask","WithMask"])

validation_set = val_datagen.flow_from_directory(Val_path,
                                                 target_size=(128, 128),
                                                 interpolation="nearest",
                                                 class_mode='binary',
                                                 classes=["WithoutMask","WithMask"])

test_set = test_datagen.flow_from_directory(Test_path,
                                            target_size = (128, 128),
                                            interpolation="nearest",
                                            class_mode='binary',
                                            classes=["WithoutMask","WithMask"])

#interpolation="nearest",
#classes=["without_mask","with_mask"]

Transforming to numpy array and getting labeled array

In [ ]:
def GetXY(gen):
    listX = []
    listY = []
    for i in range(gen.__len__()):
        gennext = gen.next()
        listX.append(gennext[0])
        listY.append(gennext[1])
    x=np.concatenate(listX)
    y=np.concatenate(listY)
    return (x,y)

In [ ]:
trainX,trainY = GetXY(training_set)
valX,valY = GetXY(validation_set)
testX,testY = GetXY(test_set)

## CNN

Setting CNN

In [ ]:
input_data = Input(shape=(128, 128, 3))

#Convolution
x = Conv2D(32, (3, 3), activation="relu")(input_data)

#Pooling
x = MaxPooling2D(pool_size = (4, 4), strides=(4, 4))(x)

#Dropout
x = Dropout(0.25)(x)

# 2nd Convolution
x = Conv2D(32, (3, 3), activation="relu")(x)

# 2nd Pooling layer
x = MaxPooling2D(pool_size = (2, 2))(x)

#Dropout
x = Dropout(0.3)(x)

#3rd Convolution
x = Conv2D(32, (3, 3), activation='relu')(x)

#3rd Pooling Layer
x = MaxPooling2D(pool_size=(2, 2))(x)

#Dropout
x = Dropout(0.3)(x)

# Flatten the layer
x = Flatten()(x)

# Fully Connected Layers
x = Dense(128, activation = 'relu')(x)
output = Dense(1, activation = 'sigmoid')(x)

cnn = Model(inputs=input_data, outputs=output)

# Compile the Neural network
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
cnn.summary()

Training the model

In [ ]:
callbacks = [ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=0.00001)]
history = cnn.fit(trainX, trainY,
                         epochs = 15,
                         verbose = 1,
                         batch_size=32,
                         validation_data = (valX, valY),
                         callbacks=callbacks)

Evaluating the model, getting report

In [ ]:
cnn.evaluate(testX, testY, verbose=2)
predict = (cnn.predict(testX) > 0.5).astype("int32")
print(classification_report(testY,predict))
print(confusion_matrix(testY, predict))

In [ ]:
print('Accuracy:', accuracy_score(testY, predict))

Сorrectly predicted images:

In [ ]:
plt.figure(figsize=(20,20))
for i in range(0, 20):
    plt.subplot(6, 10, i+1)
    number = np.random.randint(testX[testY == predict.ravel()].shape[0])
    plt.imshow(testX[testY == predict.ravel()][number])
    plt.axis("off")
plt.subplots_adjust(wspace=-0.5, hspace=1)
plt.show()

Incorrectly predicted images:

In [ ]:
plt.figure(figsize=(20,20))
for i in range(0, 20):
    plt.subplot(6, 10, i+1)
    number = np.random.randint(testX[testY != predict.ravel()].shape[0])
    plt.imshow(testX[testY != predict.ravel()][number])
    plt.axis("off")
plt.subplots_adjust(wspace=-0.5, hspace=1)
plt.show()

## Model: ResNet50 (Residual neural network)

Setting model

In [ ]:
def build_model(resnet):
    input_data = Input(shape=(128, 128, 3))
    x = resnet(input_data)
    x = GlobalMaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(128,activation='relu')(x) 
    output = Dense(1, activation='sigmoid')(x)
    
    resnet_model = Model(inputs=input_data, outputs=output)
    
    resnet_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return resnet_model

In [ ]:
#weights_path = "../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
resnet = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape = (128, 128, 3))
resnet.trainable = False
model = build_model(resnet)
model.summary()

Training the model

In [ ]:
callbacks = [ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=0.00001)]
history = model.fit(trainX, trainY,
                         epochs = 15,
                         verbose = 1,
                         batch_size=32,
                         validation_data = (valX, valY),
                         callbacks=callbacks)

Evaluating the model, getting report

In [ ]:
model.evaluate(testX, testY, verbose=2)
predict = (model.predict(testX) > 0.5).astype("int32")
print(classification_report(testY,predict))
print(confusion_matrix(testY, predict))

In [ ]:
print('Accuracy:', accuracy_score(testY, predict))

Сorrectly predicted images:

In [ ]:
plt.figure(figsize=(20,20))
for i in range(0, 20):
    plt.subplot(6, 10, i+1)
    number = np.random.randint(testX[testY == predict.ravel()].shape[0])
    plt.imshow(testX[testY == predict.ravel()][number])
    plt.axis("off")
plt.subplots_adjust(wspace=-0.5, hspace=1)
plt.show()

Incorrectly predicted images:

In [ ]:
plt.figure(figsize=(20,20))
for i in range(0, 20):
    plt.subplot(6, 10, i+1)
    number = np.random.randint(testX[testY != predict.ravel()].shape[0])
    plt.imshow(testX[testY != predict.ravel()][number])
    plt.axis("off")
plt.subplots_adjust(wspace=-0.5, hspace=1)
plt.show()